In [1]:
import yaml
import pandas as pd

try:
    with open("../../config.yaml", "r") as file:
        config = yaml.safe_load(file)

    df_business_clean = pd.read_csv(config['data']['clean_data']['business_clean'], sep=";")
    df_economy_clean = pd.read_csv(config['data']['clean_data']['economy_clean'], sep=";")

except:
    print("Yaml configuration file not found!")

In [2]:
# Combine business and economy flights
df_schedules = pd.concat([df_business_clean, df_economy_clean], ignore_index=True)
df_schedules

,airline,flight,from,to,duration,stops,dep_time,arr_time,price
0,Air India,AI-868,Delhi,Mumbai,2:00:00,0,18:00:00,20:00:00,"25,612"
1,Air India,AI-624,Delhi,Mumbai,2:15:00,0,19:00:00,21:15:00,"25,612"
2,Air India,AI-531,Delhi,Mumbai,0:45:00,1,20:00:00,20:45:00,"42,220"
3,Air India,AI-839,Delhi,Mumbai,2:30:00,1,21:25:00,23:55:00,"44,450"
4,Air India,AI-544,Delhi,Mumbai,6:40:00,1,17:15:00,23:55:00,"46,690"
...,...,...,...,...,...,...,...,...,...
300254,Vistara,UK-832,Chennai,Hyderabad,13:50:00,1,07:05:00,20:55:00,"7,697"
300255,Vistara,UK-832,Chennai,Hyderabad,13:50:00,1,07:05:00,20:55:00,"7,709"
300256,Vistara,UK-826,Chennai,Hyderabad,20:35:00,1,12:30:00,09:05:00,"8,640"
300257,Vistara,UK-822,Chennai,Hyderabad,23:20:00,1,09:45:00,09:05:00,"8,640"


In [3]:
df_schedules = df_schedules[['dep_time', 'arr_time', 'duration', 'stops']]
df_schedules

,dep_time,arr_time,duration,stops
0,18:00:00,20:00:00,2:00:00,0
1,19:00:00,21:15:00,2:15:00,0
2,20:00:00,20:45:00,0:45:00,1
3,21:25:00,23:55:00,2:30:00,1
4,17:15:00,23:55:00,6:40:00,1
...,...,...,...,...
300254,07:05:00,20:55:00,13:50:00,1
300255,07:05:00,20:55:00,13:50:00,1
300256,12:30:00,09:05:00,20:35:00,1
300257,09:45:00,09:05:00,23:20:00,1


In [4]:
%run sql_table_stops.ipynb

df_schedules = df_schedules.merge(stops_df, how='left', left_on='stops', right_on='stops')
df_schedules

,dep_time,arr_time,duration,stops,stops_id
0,18:00:00,20:00:00,2:00:00,0,1
1,19:00:00,21:15:00,2:15:00,0,1
2,20:00:00,20:45:00,0:45:00,1,2
3,21:25:00,23:55:00,2:30:00,1,2
4,17:15:00,23:55:00,6:40:00,1,2
...,...,...,...,...,...
300254,07:05:00,20:55:00,13:50:00,1,2
300255,07:05:00,20:55:00,13:50:00,1,2
300256,12:30:00,09:05:00,20:35:00,1,2
300257,09:45:00,09:05:00,23:20:00,1,2


In [5]:
df_schedules = df_schedules.drop(columns=['stops'])
df_schedules.insert(0, 'schedules_id', range(1, len(df_schedules) + 1))
df_schedules

,schedules_id,dep_time,arr_time,duration,stops_id
0,1,18:00:00,20:00:00,2:00:00,1
1,2,19:00:00,21:15:00,2:15:00,1
2,3,20:00:00,20:45:00,0:45:00,2
3,4,21:25:00,23:55:00,2:30:00,2
4,5,17:15:00,23:55:00,6:40:00,2
...,...,...,...,...,...
300254,300255,07:05:00,20:55:00,13:50:00,2
300255,300256,07:05:00,20:55:00,13:50:00,2
300256,300257,12:30:00,09:05:00,20:35:00,2
300257,300258,09:45:00,09:05:00,23:20:00,2


In [6]:
df_schedules.duplicated().any()

np.False_

In [ ]:
schedules_path = config["data"]["sql"]["schedules"]

df_schedules.to_csv(schedules_path, index=False, encoding="utf-8")